## Sampling-based vs. Anytime Planners
In this notebook, we investigate the trade-off between sampling-based and anytime planners to compare the results from an anytime planner against the sampling-based planner in combination with a post-smoothing method. They could take approximately the same time and maybe it is beneficial to have a fast planner and an efficitive post-smoothing step vs. a single anytime planner.

In [ ]:
import os
os.chdir("/home/eric/mpb/plotting")

In [ ]:
%load_ext autoreload
%autoreload 2
import mpb
from mpb import MPB, MultipleMPB
from definitions import all_planners, sampling_planners, \
                        anytime_planners, sbpl_planners

import matplotlib as mpl
mpl.rcParams['mathtext.fontset'] = 'cm'
# make sure to not use Level-3 fonts
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt

In [ ]:
pool = MultipleMPB()
for planner in sampling_planners:
    m = MPB()
    m.set_corridor_grid_env(width=150, height=150, branches=100, radius=4)
    m.set_planners([planner])
    m.set_steer_functions(['reeds_shepp'])
    m.set_smoothers(['grips', 'ompl_bspline', 'ompl_shortcut', 'ompl_simplify_max'])
    m.set_id('sampling_%s' % planner)
    pool.benchmarks.append(m)    

for planner in anytime_planners:
    for max_time in [5, 10, 15, 30, 45, 60]:
        m = MPB()
        m.set_corridor_grid_env(width=150, height=150, branches=100, radius=4)
        m.set_planners([planner])
        m.set_steer_functions(['reeds_shepp'])
        m["max_planning_time"] = max_time
        m.set_smoothers(['grips', 'ompl_bspline', 'ompl_shortcut', 'ompl_simplify_max'])
        m.set_id('anytime_%s' % planner)
        pool.benchmarks.append(m)
pool.run_parallel('sam_vs_any', runs=5)

In [ ]:
pool.visualize_trajectories(run_id='1', show_smoother=True, show_legend=True)

In [ ]:
pool.plot_planner_stats()

In [ ]:
pool.plot_smoother_stats()

In [ ]:
MPB.merge(pool.benchmarks[:4], "sampling_based.json")

In [ ]:
from plot_stats import plot_planner_stats
plot_planner_stats("sampling_based.json")

In [ ]:
from plot_stats import plot_smoother_stats
plot_smoother_stats("sampling_based.json", num_colors=5)